In [ ]:
import random
import networkx as nx
from copy import deepcopy
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
full_attack_graph = deepcopy(attack_graph)

### Assumptions

For now I assume the following
- target nodes can NOT be dropped. The defender is aware of all critical assets in his system
- Entry nodes can be dropped. The defender might not be aware of all entry points into the graph
- Intermediate nodes can be dropped.

When Plotting:
- If entry node (green) has been dropped I will colour the "new" entry nodes green instead
- If some intermediate node has been dropped and a path now ends in a "dead end", this will not be turned red, as the defender still is aware that this is not a target node.

In [ ]:
def clean_subgraph(sub_graph, original_graph):
    # initiating variables
    cleaned_graph = sub_graph.copy()
    og = original_graph

    # getting original targets 
    original_targets = []
    all_nodes = list(og.nodes())
    i = 0
    while i < len(all_nodes):
        n = all_nodes[i]
        if og.out_degree(n) == 0:
            original_targets.append(n)
        i += 1

    # existing targets in subgraph 
    existing_targets = []
    for t in original_targets:
        if t in cleaned_graph:
            existing_targets.append(t)

    # debug
    if debug_mode:
        print(f"Original legitimate targets: {original_targets}")
        print(f"Remaining legitimate targets in subgraph: {existing_targets}")

    # check if no targets remain
    if len(existing_targets) == 0:
        print("Warning: No legitimate targets remain in subgraph!")
        return cleaned_graph

    # Which nodes can reach actual targets? 
    reachable_nodes = []
    sub_nodes = list(cleaned_graph.nodes())
    k = 0
    while k < len(sub_nodes):
        node = sub_nodes[k]

        # check if node is a target
        for t in existing_targets:
            if node == t:
                reachable_nodes.append(node)
                continue  

        # check path to any target if not already added
        if node not in reachable_nodes:
            for t in existing_targets:
                try:
                    if nx.has_path(cleaned_graph, node, t):
                        reachable_nodes.append(node)
                        break
                except:
                    pass

        k += 1

    # Identify dead end branches 
    nodes_to_remove = set(cleaned_graph.nodes()) - set(reachable_nodes)

    if debug_mode:
        print(f"Removing {len(nodes_to_remove)} unreachable nodes: {nodes_to_remove}")

    # Removing dead end nodes
    nodes_to_rem_list = list(nodes_to_remove)
    d_idx = 0
    while d_idx < len(nodes_to_rem_list):
        cleaned_graph.remove_node(nodes_to_rem_list[d_idx])
        d_idx += 1

    return cleaned_graph, nodes_to_remove

In [ ]:
def create_defender_subgraph(graph, drop_percentage=0.2):
    #  initiatin vars 
    sub_graph = deepcopy(graph)

    if debug_mode:
        print(f"++++++++++")
        print(f"Start dropping & cleanup for the next subgraph with drop_percentage={drop_percentage}")

    # Identifying the target nodes
    target_nodes = []
    for n, d in sub_graph.out_degree():
        if d == 0:
            target_nodes.append(n)

    if debug_mode:
        print(f"Identified {len(target_nodes)} target nodes: {target_nodes}")

    # Creating a list of non-target nodes 
    droppable_nodes = []
    for curr in sub_graph.nodes():  
        is_target = False
        i = 0
        while i < len(target_nodes):
            if target_nodes[i] == curr:
                is_target = True
                break
            i += 1
        if not is_target:
            droppable_nodes.append(curr)

    # Calculating how many nodes to drop
    num_to_drop = int(len(droppable_nodes) * drop_percentage)
    if drop_percentage > 0 and num_to_drop == 0:
        num_to_drop = 1

    # Randomly selecting nodes to drop using shuffle
    dropped_nodes = []
    if droppable_nodes:
        temp_list = list(droppable_nodes)
        random.shuffle(temp_list)
        y = 0
        while y < num_to_drop and y < len(temp_list):
            dropped_nodes.append(temp_list[y])
            y += 1

    if debug_mode:
        print(f"Dropping {len(dropped_nodes)} nodes: {dropped_nodes}")

    # Removing 
    i = 0
    while i < len(dropped_nodes):
        sub_graph.remove_node(dropped_nodes[i])
        i += 1

    if debug_mode:
        print(f"Original graph had {len(graph.nodes())} nodes, subgraph has {len(sub_graph.nodes())} nodes")

    # Removing dead branches
    sub_graph, cleaned_nodes = clean_subgraph(sub_graph, graph)

    return sub_graph, dropped_nodes, cleaned_nodes

In [ ]:
# Function to generate subgraphs for a specific drop percentage
def generate_defender_subgraphs(graph, num_subgraphs=100, drop_percentage=0.2):
    """
    Generates a list of defender subgraphs using the specified drop percentage.
    """
    if debug_mode:
        print(f"Generating {num_subgraphs} subgraphs with drop_percentage={drop_percentage}")
    return [create_defender_subgraph(graph, drop_percentage) for _ in range(num_subgraphs)]

In [ ]:
try:
    if drop_pct is None:
        current_drop_percentage = 0.2
    else:
        current_drop_percentage = drop_pct
except NameError:
    # Variable doesn't exist yet
    current_drop_percentage = 0.2


try:
    if number_of_generated_subgraphs is None: 
        number_of_generated_subgraphs = 100
    else:
        number_of_generated_subgraphs = number_of_generated_subgraphs
except NameError:
    # Variable doesn't exist yet
    number_of_generated_subgraphs = 100

In [ ]:
# Generate subgraphs with the given drop percentage
defender_subgraphs_list = generate_defender_subgraphs(
    full_attack_graph, 
    num_subgraphs=number_of_generated_subgraphs,
    drop_percentage=current_drop_percentage
)